<a href="https://colab.research.google.com/github/PuranikPranav/Network-Models/blob/main/Last_mile_logistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 18.8 MB/s eta 0:00:00


In [ ]:
from scx.optimize import Model

In [ ]:
## Type 1 Network Design - 2 Kitchens and 5 Delivery Areas
##Standard Transportation Model
#Available delivery time Data
delivery_time_1 = [
    {
        'origin':'K1',
        'destination':'A1',
        'time':17
    },
    {
        'origin':'K1',
        'destination':'A2',
        'time':5
    },
    {
        'origin':'K1',
        'destination':'A3',
        'time':16
    },
    {
        'origin':'K1',
        'destination':'A4',
        'time':16
    },
    {
        'origin':'K1',
        'destination':'A5',
        'time':12
    },
    {
        'origin':'K2',
        'destination':'A1',
        'time':7
    },
    {
        'origin':'K2',
        'destination':'A2',
        'time':8
    },
    {
        'origin':'K2',
        'destination':'A3',
        'time':19
    },
    {
        'origin':'K2',
        'destination':'A4',
        'time':5
    },
    {
        'origin':'K2',
        'destination':'A5',
        'time':15
    }

]

#Demand for each area
demand = [
    {
        'area':'A1',
        'demand':101
    },
    {
        'area':'A2',
        'demand':61
    },
    {
        'area':'A3',
        'demand':101
    },
    {
        'area':'A4',
        'demand':87
    },
    {
        'area':'A5',
        'demand':50
    }

]

#Kitchen capacities
capacity = [
    {
        'kitchen':'K1',
        'capacity':250
    },
    {
        'kitchen':'K2',
        'capacity':350
    }

]

In [ ]:
for d in delivery_time_1:
  #Create decision variables
  d['amt'] = Model.variable(name=f"{d['origin']}__{d['destination']}__amt",lowBound=0)

In [ ]:
#Initialize the model
my_model = Model(name="Food on Wheels Type 1", sense = "minimize")

#Add the objective function
my_model.add_objective(fn=Model.sum([d['amt']*d['time'] for d in delivery_time_1]))

#Add the demand constraints
for x in demand:
  my_model.add_constraint(
      name=f"{x['area']}__demand",
      fn=Model.sum([d['amt'] for d in delivery_time_1 if d['destination'] == x['area']]) >= x['demand']
  )

#Add the capacity constraints
for x in capacity:
  my_model.add_constraint(
      name = f"{x['kitchen']}__capacity",
      fn = Model.sum([d['amt'] for d in delivery_time_1 if d['origin'] == x['kitchen']]) <= x['capacity']
  )

#Solve the model
my_model.solve()

/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [ ]:
my_model.show_formulation()

Food_on_Wheels_Type_1:
MINIMIZE
17*K1__A1__amt + 5*K1__A2__amt + 16*K1__A3__amt + 16*K1__A4__amt + 12*K1__A5__amt + 7*K2__A1__amt + 8*K2__A2__amt + 19*K2__A3__amt + 5*K2__A4__amt + 15*K2__A5__amt + 0
SUBJECT TO
A1__demand: K1__A1__amt + K2__A1__amt >= 101

A2__demand: K1__A2__amt + K2__A2__amt >= 61

A3__demand: K1__A3__amt + K2__A3__amt >= 101

A4__demand: K1__A4__amt + K2__A4__amt >= 87

A5__demand: K1__A5__amt + K2__A5__amt >= 50

K1__capacity: K1__A1__amt + K1__A2__amt + K1__A3__amt + K1__A4__amt
 + K1__A5__amt <= 250

K2__capacity: K2__A1__amt + K2__A2__amt + K2__A3__amt + K2__A4__amt
 + K2__A5__amt <= 350

VARIABLES
K1__A1__amt Continuous
K1__A2__amt Continuous
K1__A3__amt Continuous
K1__A4__amt Continuous
K1__A5__amt Continuous
K2__A1__amt Continuous
K2__A2__amt Continuous
K2__A3__amt Continuous
K2__A4__amt Continuous
K2__A5__amt Continuous



In [ ]:
#Show the outputs
my_model.show_outputs()

{'objective': 3663.0,
 'status': 'Optimal',
 'variables': {'K1__A1__amt': 0.0,
               'K1__A2__amt': 61.0,
               'K1__A3__amt': 101.0,
               'K1__A4__amt': 0.0,
               'K1__A5__amt': 50.0,
               'K2__A1__amt': 101.0,
               'K2__A2__amt': 0.0,
               'K2__A3__amt': 0.0,
               'K2__A4__amt': 87.0,
               'K2__A5__amt': 0.0}}


In [ ]:
## Type 2 Network Model --> 1 Central Kitchen, 3 Distribution Hubs, 5 Delivery Areas
##Standard Transhipment Data
# Available Data

delivery_time_2 = [
    {
        'source':'Central_Kitchen',
        'destination':'Hub_1',
        'time':2
    },
    {
        'source':'Central_Kitchen',
        'destination':'Hub_2',
        'time':5
    },
    {
        'source':'Central_Kitchen',
        'destination':'Hub_3',
        'time':7
    },
    {
        'source':'Hub_1',
        'destination':'A1',
        'time':13
    },
    {
        'source':'Hub_1',
        'destination':'A2',
        'time':7
    },
    {
        'source':'Hub_1',
        'destination':'A3',
        'time':14
    },
    {
        'source':'Hub_1',
        'destination':'A4',
        'time':14
    },
    {
        'source':'Hub_1',
        'destination':'A5',
        'time':5
    },
    {
        'source':'Hub_2',
        'destination':'A1',
        'time':15
    },
    {
        'source':'Hub_2',
        'destination':'A2',
        'time':4
    },
    {
        'source':'Hub_2',
        'destination':'A3',
        'time':10
    },
    {
        'source':'Hub_2',
        'destination':'A4',
        'time':8
    },
    {
        'source':'Hub_2',
        'destination':'A5',
        'time':7
    },
    {
        'source':'Hub_3',
        'destination':'A1',
        'time':3
    },
    {
        'source':'Hub_3',
        'destination':'A2',
        'time':3
    },
    {
        'source':'Hub_3',
        'destination':'A3',
        'time':11
    },
    {
        'source':'Hub_3',
        'destination':'A4',
        'time':11
    },
    {
        'source':'Hub_3',
        'destination':'A5',
        'time':10
    }
]

#Demand for each area
demand = [
    {
        'area':'A1',
        'demand':101
    },
    {
        'area':'A2',
        'demand':61
    },
    {
        'area':'A3',
        'demand':101
    },
    {
        'area':'A4',
        'demand':87
    },
    {
        'area':'A5',
        'demand':50
    }

]

#Capacity for each new DC
DC_capacity = [
    {
        'DC':'Hub_1',
        'capacity': 150
    },
    {
        'DC':'Hub_2',
        'capacity': 100
    },
    {
        'DC':'Hub_3',
        'capacity': 150
    }
]



In [ ]:
for d in delivery_time_2:
  #Create decision variables
  d['amt'] = Model.variable(name=f"{d['source']}__{d['destination']}__amt",lowBound=0)

In [ ]:
#Initialize the model
my_model = Model(name="Food on Wheels Type 2", sense = "minimize")

#Add the objective function
my_model.add_objective(fn=Model.sum([d['amt']*d['time'] for d in delivery_time_2]))

#Add the demand constraints
for x in demand:
  my_model.add_constraint(
      name=f"{x['area']}__demand",
      fn=Model.sum([d['amt'] for d in delivery_time_2 if d['destination'] == x['area']]) >= x['demand']
  )


##Add the capacity constraints
for x in DC_capacity:
  my_model.add_constraint(
      name = f"{x['DC']}__capacity",
      fn = Model.sum([d['amt'] for d in delivery_time_2 if d['source'] == x['DC']]) <= x['capacity']
  )


#Add the conservation of flow constraint
for x in DC_capacity:
  my_model.add_constraint(
      name = f"{x['DC']}__COF",
      fn = Model.sum([d['amt'] for d in delivery_time_2 if d['destination'] == x['DC']]) == Model.sum([d['amt'] for d in delivery_time_2 if d['source'] == x['DC']])
  )

#Solve the model
my_model.solve()

In [ ]:
my_model.show_formulation()

Food_on_Wheels_Type_2:
MINIMIZE
2*Central_Kitchen__Hub_1__amt + 5*Central_Kitchen__Hub_2__amt + 7*Central_Kitchen__Hub_3__amt + 13*Hub_1__A1__amt + 7*Hub_1__A2__amt + 14*Hub_1__A3__amt + 14*Hub_1__A4__amt + 5*Hub_1__A5__amt + 15*Hub_2__A1__amt + 4*Hub_2__A2__amt + 10*Hub_2__A3__amt + 8*Hub_2__A4__amt + 7*Hub_2__A5__amt + 3*Hub_3__A1__amt + 3*Hub_3__A2__amt + 11*Hub_3__A3__amt + 11*Hub_3__A4__amt + 10*Hub_3__A5__amt + 0
SUBJECT TO
A1__demand: Hub_1__A1__amt + Hub_2__A1__amt + Hub_3__A1__amt >= 101

A2__demand: Hub_1__A2__amt + Hub_2__A2__amt + Hub_3__A2__amt >= 61

A3__demand: Hub_1__A3__amt + Hub_2__A3__amt + Hub_3__A3__amt >= 101

A4__demand: Hub_1__A4__amt + Hub_2__A4__amt + Hub_3__A4__amt >= 87

A5__demand: Hub_1__A5__amt + Hub_2__A5__amt + Hub_3__A5__amt >= 50

Hub_1__capacity: Hub_1__A1__amt + Hub_1__A2__amt + Hub_1__A3__amt
 + Hub_1__A4__amt + Hub_1__A5__amt <= 150

Hub_2__capacity: Hub_2__A1__amt + Hub_2__A2__amt + Hub_2__A3__amt
 + Hub_2__A4__amt + Hub_2__A5__amt <= 100

Hub_3_

In [ ]:
#Show the outputs
my_model.show_outputs()

{'objective': 4692.0,
 'status': 'Optimal',
 'variables': {'Central_Kitchen__Hub_1__amt': 150.0,
               'Central_Kitchen__Hub_2__amt': 100.0,
               'Central_Kitchen__Hub_3__amt': 150.0,
               'Hub_1__A1__amt': 0.0,
               'Hub_1__A2__amt': 12.0,
               'Hub_1__A3__amt': 88.0,
               'Hub_1__A4__amt': 0.0,
               'Hub_1__A5__amt': 50.0,
               'Hub_2__A1__amt': 0.0,
               'Hub_2__A2__amt': 0.0,
               'Hub_2__A3__amt': 13.0,
               'Hub_2__A4__amt': 87.0,
               'Hub_2__A5__amt': 0.0,
               'Hub_3__A1__amt': 101.0,
               'Hub_3__A2__amt': 49.0,
               'Hub_3__A3__amt': 0.0,
               'Hub_3__A4__amt': 0.0,
               'Hub_3__A5__amt': 0.0}}


In [ ]:
## Type 3 Network Model --> 1 Central Kitchen, 3 Distribution Hubs, 5 Delivery Areas - Select one from 3 more candidate DCs
##Standard Transhipment Data with Binary variables
# Available Data

delivery_time_3 = [
    {
        'source':'Central_Kitchen',
        'destination':'Hub_1',
        'time':2
    },
    {
        'source':'Central_Kitchen',
        'destination':'Hub_2',
        'time':5
    },
    {
        'source':'Central_Kitchen',
        'destination':'Hub_3',
        'time':7
    },
    {
        'source':'Central_Kitchen',
        'destination':'Hub_4',
        'time':5
    },
    {
        'source':'Central_Kitchen',
        'destination':'Hub_5',
        'time':6
    },
    {
        'source':'Central_Kitchen',
        'destination':'Hub_6',
        'time':4
    },
    {
        'source':'Hub_1',
        'destination':'A1',
        'time':13
    },
    {
        'source':'Hub_1',
        'destination':'A2',
        'time':7
    },
    {
        'source':'Hub_1',
        'destination':'A3',
        'time':14
    },
    {
        'source':'Hub_1',
        'destination':'A4',
        'time':14
    },
    {
        'source':'Hub_1',
        'destination':'A5',
        'time':5
    },
    {
        'source':'Hub_2',
        'destination':'A1',
        'time':15
    },
    {
        'source':'Hub_2',
        'destination':'A2',
        'time':4
    },
    {
        'source':'Hub_2',
        'destination':'A3',
        'time':10
    },
    {
        'source':'Hub_2',
        'destination':'A4',
        'time':8
    },
    {
        'source':'Hub_2',
        'destination':'A5',
        'time':7
    },
    {
        'source':'Hub_3',
        'destination':'A1',
        'time':3
    },
    {
        'source':'Hub_3',
        'destination':'A2',
        'time':3
    },
    {
        'source':'Hub_3',
        'destination':'A3',
        'time':11
    },
    {
        'source':'Hub_3',
        'destination':'A4',
        'time':11
    },
    {
        'source':'Hub_3',
        'destination':'A5',
        'time':10
    },
    {
        'source':'Hub_4',
        'destination':'A1',
        'time':10
    },
    {
        'source':'Hub_4',
        'destination':'A2',
        'time':13
    },
    {
        'source':'Hub_4',
        'destination':'A3',
        'time':14
    },
    {
        'source':'Hub_4',
        'destination':'A4',
        'time':8
    },
    {
        'source':'Hub_4',
        'destination':'A5',
        'time':12
    },
    {
        'source':'Hub_5',
        'destination':'A1',
        'time':5
    },
    {
        'source':'Hub_5',
        'destination':'A2',
        'time':8
    },
    {
        'source':'Hub_5',
        'destination':'A3',
        'time':4
    },
    {
        'source':'Hub_5',
        'destination':'A4',
        'time':9
    },
    {
        'source':'Hub_5',
        'destination':'A5',
        'time':11
    },
    {
        'source':'Hub_6',
        'destination':'A1',
        'time':13
    },
    {
        'source':'Hub_6',
        'destination':'A2',
        'time':5
    },
    {
        'source':'Hub_6',
        'destination':'A3',
        'time':8
    },
    {
        'source':'Hub_6',
        'destination':'A4',
        'time':7
    },
    {
        'source':'Hub_6',
        'destination':'A5',
        'time':10
    }
]

#Demand for each area
demand = [
    {
        'area':'A1',
        'demand':101
    },
    {
        'area':'A2',
        'demand':61
    },
    {
        'area':'A3',
        'demand':101
    },
    {
        'area':'A4',
        'demand':87
    },
    {
        'area':'A5',
        'demand':50
    }

]

#Capacity for each new DC
DC = [
    {
        'DC':'Hub_1',
        'capacity': 150,
        'type' : 'owned'
    },
    {
        'DC':'Hub_2',
        'capacity': 100,
        'type' : 'owned'
    },
    {
        'DC':'Hub_3',
        'capacity': 150,
        'type':'owned'
    },
    {
        'DC':'Hub_4',
        'capacity': 200,
        'type':'rental'
    },
    {
        'DC':'Hub_5',
        'capacity': 200,
        'type':'rental'
    },
    {
        'DC':'Hub_6',
        'capacity': 200,
        'type':'rental'
    }
]



In [ ]:
for d in delivery_time_3:
  #Create decision variables
  d['amt'] = Model.variable(name=f"{d['source']}__{d['destination']}__amt",lowBound=0)

for d in DC:
  #Create binary variables
  d['rent'] = Model.variable(name=f"{d['DC']}__rent", cat='Binary')

M = 999999

In [ ]:
#Initialize the model
my_model = Model(name="Food on Wheels Type 3", sense = "minimize")

#Add the objective function
my_model.add_objective(fn=Model.sum([d['amt']*d['time'] for d in delivery_time_3]))

#Add the demand constraints
for x in demand:
  my_model.add_constraint(
      name=f"{x['area']}__demand",
      fn=Model.sum([d['amt'] for d in delivery_time_3 if d['destination'] == x['area']]) >= x['demand']
  )


##Add the capacity constraints
for x in DC:
  my_model.add_constraint(
      name = f"{x['DC']}__capacity",
      fn = Model.sum([d['amt'] for d in delivery_time_3 if d['source'] == x['DC']]) <= x['capacity']
  )
  #Add the conservation of flow constraint
  my_model.add_constraint(
      name = f"{x['DC']}__COF",
      fn = Model.sum([d['amt'] for d in delivery_time_3 if d['destination'] == x['DC']]) == Model.sum([d['amt'] for d in delivery_time_3 if d['source'] == x['DC']])
  )
  my_model.add_constraint(
      name = f"{x['DC']}__Linking",
      fn = Model.sum([d['amt'] for d in delivery_time_3 if d['source'] == x['DC']]) <= x['rent']*M
  )

my_model.add_constraint(
    name = f"Open one hub",
    fn = Model.sum([d['rent'] for d in DC if d['type'] == 'rental']) == 1
)

#Solve the model
my_model.solve()

In [ ]:
my_model.show_formulation()

Food_on_Wheels_Type_3:
MINIMIZE
2*Central_Kitchen__Hub_1__amt + 5*Central_Kitchen__Hub_2__amt + 7*Central_Kitchen__Hub_3__amt + 5*Central_Kitchen__Hub_4__amt + 6*Central_Kitchen__Hub_5__amt + 4*Central_Kitchen__Hub_6__amt + 13*Hub_1__A1__amt + 7*Hub_1__A2__amt + 14*Hub_1__A3__amt + 14*Hub_1__A4__amt + 5*Hub_1__A5__amt + 15*Hub_2__A1__amt + 4*Hub_2__A2__amt + 10*Hub_2__A3__amt + 8*Hub_2__A4__amt + 7*Hub_2__A5__amt + 3*Hub_3__A1__amt + 3*Hub_3__A2__amt + 11*Hub_3__A3__amt + 11*Hub_3__A4__amt + 10*Hub_3__A5__amt + 10*Hub_4__A1__amt + 13*Hub_4__A2__amt + 14*Hub_4__A3__amt + 8*Hub_4__A4__amt + 12*Hub_4__A5__amt + 5*Hub_5__A1__amt + 8*Hub_5__A2__amt + 4*Hub_5__A3__amt + 9*Hub_5__A4__amt + 11*Hub_5__A5__amt + 13*Hub_6__A1__amt + 5*Hub_6__A2__amt + 8*Hub_6__A3__amt + 7*Hub_6__A4__amt + 10*Hub_6__A5__amt + 0
SUBJECT TO
A1__demand: Hub_1__A1__amt + Hub_2__A1__amt + Hub_3__A1__amt + Hub_4__A1__amt
 + Hub_5__A1__amt + Hub_6__A1__amt >= 101

A2__demand: Hub_1__A2__amt + Hub_2__A2__amt + Hub_3__A2__

In [ ]:
my_model.show_outputs()

{'objective': 4050.0,
 'status': 'Optimal',
 'variables': {'Central_Kitchen__Hub_1__amt': 111.0,
               'Central_Kitchen__Hub_2__amt': 87.0,
               'Central_Kitchen__Hub_3__amt': 101.0,
               'Central_Kitchen__Hub_4__amt': 0.0,
               'Central_Kitchen__Hub_5__amt': 101.0,
               'Central_Kitchen__Hub_6__amt': 0.0,
               'Hub_1__A1__amt': 0.0,
               'Hub_1__A2__amt': 61.0,
               'Hub_1__A3__amt': 0.0,
               'Hub_1__A4__amt': 0.0,
               'Hub_1__A5__amt': 50.0,
               'Hub_1__rent': 1.0,
               'Hub_2__A1__amt': 0.0,
               'Hub_2__A2__amt': 0.0,
               'Hub_2__A3__amt': 0.0,
               'Hub_2__A4__amt': 87.0,
               'Hub_2__A5__amt': 0.0,
               'Hub_2__rent': 1.0,
               'Hub_3__A1__amt': 101.0,
               'Hub_3__A2__amt': 0.0,
               'Hub_3__A3__amt': 0.0,
               'Hub_3__A4__amt': 0.0,
               'Hub_3__A5__amt': 0.0

In [ ]:
##Type 4 - Including Level of service constraint
#Atleast 75% of Demand should be delivered under 7 minutes

## Type 3 Network Model --> 1 Central Kitchen, 3 Distribution Hubs, 5 Delivery Areas - Select one from 3 more candidate DCs
##Standard Transhipment Data with Binary variables
# Available Data

delivery_time_4 = [
    {
        'source':'Central_Kitchen',
        'destination':'Hub_1',
        'time':2
    },
    {
        'source':'Central_Kitchen',
        'destination':'Hub_2',
        'time':5
    },
    {
        'source':'Central_Kitchen',
        'destination':'Hub_3',
        'time':7
    },
    {
        'source':'Central_Kitchen',
        'destination':'Hub_4',
        'time':5
    },
    {
        'source':'Central_Kitchen',
        'destination':'Hub_5',
        'time':6
    },
    {
        'source':'Central_Kitchen',
        'destination':'Hub_6',
        'time':4
    },
    {
        'source':'Hub_1',
        'destination':'A1',
        'time':13
    },
    {
        'source':'Hub_1',
        'destination':'A2',
        'time':7
    },
    {
        'source':'Hub_1',
        'destination':'A3',
        'time':14
    },
    {
        'source':'Hub_1',
        'destination':'A4',
        'time':14
    },
    {
        'source':'Hub_1',
        'destination':'A5',
        'time':5
    },
    {
        'source':'Hub_2',
        'destination':'A1',
        'time':15
    },
    {
        'source':'Hub_2',
        'destination':'A2',
        'time':4
    },
    {
        'source':'Hub_2',
        'destination':'A3',
        'time':10
    },
    {
        'source':'Hub_2',
        'destination':'A4',
        'time':8
    },
    {
        'source':'Hub_2',
        'destination':'A5',
        'time':7
    },
    {
        'source':'Hub_3',
        'destination':'A1',
        'time':3
    },
    {
        'source':'Hub_3',
        'destination':'A2',
        'time':3
    },
    {
        'source':'Hub_3',
        'destination':'A3',
        'time':11
    },
    {
        'source':'Hub_3',
        'destination':'A4',
        'time':11
    },
    {
        'source':'Hub_3',
        'destination':'A5',
        'time':10
    },
    {
        'source':'Hub_4',
        'destination':'A1',
        'time':10
    },
    {
        'source':'Hub_4',
        'destination':'A2',
        'time':13
    },
    {
        'source':'Hub_4',
        'destination':'A3',
        'time':14
    },
    {
        'source':'Hub_4',
        'destination':'A4',
        'time':8
    },
    {
        'source':'Hub_4',
        'destination':'A5',
        'time':12
    },
    {
        'source':'Hub_5',
        'destination':'A1',
        'time':5
    },
    {
        'source':'Hub_5',
        'destination':'A2',
        'time':8
    },
    {
        'source':'Hub_5',
        'destination':'A3',
        'time':4
    },
    {
        'source':'Hub_5',
        'destination':'A4',
        'time':9
    },
    {
        'source':'Hub_5',
        'destination':'A5',
        'time':11
    },
    {
        'source':'Hub_6',
        'destination':'A1',
        'time':13
    },
    {
        'source':'Hub_6',
        'destination':'A2',
        'time':5
    },
    {
        'source':'Hub_6',
        'destination':'A3',
        'time':8
    },
    {
        'source':'Hub_6',
        'destination':'A4',
        'time':7
    },
    {
        'source':'Hub_6',
        'destination':'A5',
        'time':10
    }
]

#Demand for each area
demand = [
    {
        'area':'A1',
        'demand':101
    },
    {
        'area':'A2',
        'demand':61
    },
    {
        'area':'A3',
        'demand':101
    },
    {
        'area':'A4',
        'demand':87
    },
    {
        'area':'A5',
        'demand':50
    }

]

#Capacity for each new DC
DC = [
    {
        'DC':'Hub_1',
        'capacity': 150,
        'type' : 'owned'
    },
    {
        'DC':'Hub_2',
        'capacity': 100,
        'type' : 'owned'
    },
    {
        'DC':'Hub_3',
        'capacity': 150,
        'type':'owned'
    },
    {
        'DC':'Hub_4',
        'capacity': 200,
        'type':'rental'
    },
    {
        'DC':'Hub_5',
        'capacity': 200,
        'type':'rental'
    },
    {
        'DC':'Hub_6',
        'capacity': 200,
        'type':'rental'
    }
]


In [ ]:
for d in delivery_time_4:
  #Create decision variables
  d['amt'] = Model.variable(name=f"{d['source']}__{d['destination']}__amt",lowBound=0)

for d in DC:
  #Create binary variables
  d['rent'] = Model.variable(name=f"{d['DC']}__rent", cat='Binary')

M = 999999

In [ ]:
#Initialize the model
my_model = Model(name="Food on Wheels Type 3", sense = "minimize")

#Add the objective function
my_model.add_objective(fn=Model.sum([d['amt']*d['time'] for d in delivery_time_4]))

#Add the demand constraints
for x in demand:
  my_model.add_constraint(
      name=f"{x['area']}__demand",
      fn=Model.sum([d['amt'] for d in delivery_time_4 if d['destination'] == x['area']]) >= x['demand']
  )


##Add the capacity constraints
for x in DC:
  my_model.add_constraint(
      name = f"{x['DC']}__capacity",
      fn = Model.sum([d['amt'] for d in delivery_time_4 if d['source'] == x['DC']]) <= x['capacity']
  )
  #Add the conservation of flow constraint
  my_model.add_constraint(
      name = f"{x['DC']}__COF",
      fn = Model.sum([d['amt'] for d in delivery_time_4 if d['destination'] == x['DC']]) == Model.sum([d['amt'] for d in delivery_time_4 if d['source'] == x['DC']])
  )
  my_model.add_constraint(
      name = f"{x['DC']}__Linking",
      fn = Model.sum([d['amt'] for d in delivery_time_4 if d['source'] == x['DC']]) <= x['rent']*M
  )

my_model.add_constraint(
    name = f"Open one hub",
    fn = Model.sum([d['rent'] for d in DC if d['type'] == 'rental']) == 1
)

#Add the Level of service constraint
areas = [d['area'] for d in demand]
my_model.add_constraint(
      name=f"Level of service constraint",
      fn=Model.sum([d['amt'] for d in delivery_time_4 if d['destination'] in areas and d['time'] < 7]) >= Model.sum([d['demand'] for d in demand])*0.75
  )

#Solve the model
my_model.solve()

/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [ ]:
my_model.show_formulation()

Food_on_Wheels_Type_3:
MINIMIZE
2*Central_Kitchen__Hub_1__amt + 5*Central_Kitchen__Hub_2__amt + 7*Central_Kitchen__Hub_3__amt + 5*Central_Kitchen__Hub_4__amt + 6*Central_Kitchen__Hub_5__amt + 4*Central_Kitchen__Hub_6__amt + 13*Hub_1__A1__amt + 7*Hub_1__A2__amt + 14*Hub_1__A3__amt + 14*Hub_1__A4__amt + 5*Hub_1__A5__amt + 15*Hub_2__A1__amt + 4*Hub_2__A2__amt + 10*Hub_2__A3__amt + 8*Hub_2__A4__amt + 7*Hub_2__A5__amt + 3*Hub_3__A1__amt + 3*Hub_3__A2__amt + 11*Hub_3__A3__amt + 11*Hub_3__A4__amt + 10*Hub_3__A5__amt + 10*Hub_4__A1__amt + 13*Hub_4__A2__amt + 14*Hub_4__A3__amt + 8*Hub_4__A4__amt + 12*Hub_4__A5__amt + 5*Hub_5__A1__amt + 8*Hub_5__A2__amt + 4*Hub_5__A3__amt + 9*Hub_5__A4__amt + 11*Hub_5__A5__amt + 13*Hub_6__A1__amt + 5*Hub_6__A2__amt + 8*Hub_6__A3__amt + 7*Hub_6__A4__amt + 10*Hub_6__A5__amt + 0
SUBJECT TO
A1__demand: Hub_1__A1__amt + Hub_2__A1__amt + Hub_3__A1__amt + Hub_4__A1__amt
 + Hub_5__A1__amt + Hub_6__A1__amt >= 101

A2__demand: Hub_1__A2__amt + Hub_2__A2__amt + Hub_3__A2__

In [ ]:
#Print the output
my_model.show_outputs()

{'objective': 4085.0,
 'status': 'Optimal',
 'variables': {'Central_Kitchen__Hub_1__amt': 63.0,
               'Central_Kitchen__Hub_2__amt': 100.0,
               'Central_Kitchen__Hub_3__amt': 136.0,
               'Central_Kitchen__Hub_4__amt': 0.0,
               'Central_Kitchen__Hub_5__amt': 101.0,
               'Central_Kitchen__Hub_6__amt': 0.0,
               'Hub_1__A1__amt': 0.0,
               'Hub_1__A2__amt': 13.0,
               'Hub_1__A3__amt': 0.0,
               'Hub_1__A4__amt': 0.0,
               'Hub_1__A5__amt': 50.0,
               'Hub_1__rent': 1.0,
               'Hub_2__A1__amt': 0.0,
               'Hub_2__A2__amt': 13.0,
               'Hub_2__A3__amt': 0.0,
               'Hub_2__A4__amt': 87.0,
               'Hub_2__A5__amt': 0.0,
               'Hub_2__rent': 1.0,
               'Hub_3__A1__amt': 101.0,
               'Hub_3__A2__amt': 35.0,
               'Hub_3__A3__amt': 0.0,
               'Hub_3__A4__amt': 0.0,
               'Hub_3__A5__amt': 0